In [2]:
import pandas as pd
import math

# Read the CSV file
df1 = pd.read_csv("bus.csv", header=0)
df2 = pd.read_csv("train.csv", header=0)
df3 = pd.read_csv("tram.csv", header=0)

df_combined = pd.concat([df1,df2,df3])
df_combined = df_combined[[' STOP_NAME']]
df_combined[' STOP_NAME'] = df_combined[' STOP_NAME'].apply(lambda x: x.split('(')[1][:-1].lower())
# df_combined = df_combined.groupby(' STOP_NAME').size().reset_index(name = 'counts')
df_combined

,STOP_NAME
0,st albans
1,st albans
2,st albans
3,st albans
4,st albans
...,...
1660,camberwell
1661,surrey hills
1662,surrey hills
1663,surrey hills


In [3]:
df4 = pd.read_csv("regional_bus.csv", header=0)
df5 = pd.read_csv("regional_train.csv", header=0)
df6 = pd.read_csv("regional_couch.csv", header=0)

def processData(data):
    clean = ''
    if('(' not in data):
        clean = data.lower()
    else:
        count = data.count('(')
        if(count == 1):
            clean = data.split('(')[1][:-1].lower()
        elif(count == 2):
            indexFirst = data.find('(')
            indexFirst2 = data.find(')')
            indexLast = data.rfind('(')
            indexLast2 = data.find(')')
            if(indexFirst2 > indexLast):
                clean = data[indexFirst+1:indexLast-1].lower()
            else:
                clean = data.split('(')[2][:-1].lower()

        else:
            clean =  data.split('(')[2][:-1].lower()
    return clean

df_combined2 = pd.concat([df4,df5,df6])
df_combined2 = df_combined2[[' STOP_NAME']]
df_combined2[' STOP_NAME'] = df_combined2[' STOP_NAME'].apply(lambda x: processData(x))
df_combined3 = pd.concat([df_combined,df_combined2])
df_combined3 = df_combined3.groupby(' STOP_NAME').size().reset_index(name = 'counts')
print(df_combined3['counts'].sum())
df_combined3


24539


,STOP_NAME,counts
0,abbotsford,21
1,aberfeldie,18
2,aintree,19
3,aireys inlet,8
4,airport west,60
...,...,...
1082,yea,1
1083,yendon,2
1084,yinnar,5
1085,yulecart,2


In [4]:
import json

ssa_code = open('../vic_sal/geo_vic_filter_lga.geojson')
ssa_code_json = json.load(ssa_code)
ssa_code_json = ssa_code_json['features']
ssa_code_json[0]

{'type': 'Feature',
 'geometry': {'coordinates': [[[143.62891925800008, -38.02339335099998],
    [143.62986219800007, -38.02783601099998],
    [143.6275638280001, -38.02986503099993],
    [143.6211395790001, -38.03226854099995],
    [143.62169276800012, -38.036789160999945],
    [143.6175827190001, -38.03756738099997],
    [143.61767236900005, -38.03327867099995],
    [143.6122083900001, -38.03477217999995],
    [143.61223409000002, -38.03894485],
    [143.61552685900006, -38.0413263],
    [143.61481583800003, -38.04542328999997],
    [143.61228536900012, -38.04718924999997],
    [143.61281720800002, -38.05336496999996],
    [143.618518837, -38.05346080099997],
    [143.615514447, -38.05963152099997],
    [143.6160717670001, -38.061357090999934],
    [143.62131221500007, -38.064359140999954],
    [143.62536108400002, -38.069349140999975],
    [143.63053105300003, -38.07231589199995],
    [143.6322864120001, -38.07720985199995],
    [143.63581764100002, -38.080197581999975],
    [143.63

In [5]:
transportation_geojson = {'type':'FeatureCollection','features':[]}

for index, data in df_combined3.iterrows():
    name = data[' STOP_NAME']

    matchData = next((item for item in ssa_code_json for s in item["properties"]['ssa_name'] if name in s),None)

    # if(matchData != None):
    #     resultFeature = {'type':'Feature','geometry':matchData['geometry'],'properties': matchData['properties']}
    #     if('transportation_number' not in resultFeature['properties']):
    #         resultFeature['properties']['transportation_number'] = data['counts']
    #     else:
    #         resultFeature['properties']['transportation_number'] += data['counts']
    # transportation_geojson['features'].append(resultFeature)

    if(matchData != None):
        resultFeature = [data for data in transportation_geojson['features'] for sub in data['properties']['ssa_name'] if name in sub]
        if(len(resultFeature) != 0):
            resultFeature[0]['properties']['transportation_number'] += data['counts']
        else:
            resultFeature = {'type':'Feature','geometry':matchData['geometry'],'properties': matchData['properties']}
            resultFeature['properties']['transportation_number'] = data['counts']
            transportation_geojson['features'].append(resultFeature)

transportation_geojson

with open('transportation_geojson.geojson', 'w') as fp:
    json.dump(transportation_geojson, fp)
    